In [1]:
import ibmdata
import pandas as pd
import datetime
import plotly.graph_objects as go
import numpy as np
import kaleido

In [2]:
DEFAULT_DAYS_BACK = 600
DEFAULT_END_DATE = datetime.datetime.today()
DEFAULT_START_DATE = (DEFAULT_END_DATE - datetime.timedelta(days=DEFAULT_DAYS_BACK)).date()
DEFAULT_END_DATE = (DEFAULT_END_DATE + datetime.timedelta(days=1)).date()

ALL_FAMILY_CODES = "'M1'"

# Possibly Useful Functions for Generating SQL

In [3]:
def convert_list_to_SQL_input(mylist):
    out = ""
    for i,l in enumerate(mylist):
        if i != len(mylist)-1:
            out += "'" + l + "', "
        else:
            out += "'" + l + "'"
    return out

2023-11-25 03:51:46,707 INFO     ibmdata   : version 0.10.0 of ibmdata is available for install, you're using 0.9.8


In [4]:
def generate_SQL_GroupBy(mydict, parm_label = 'parm_Label', val = 'parmYield', agg = 'AVG'):
    output = ''
    n = len(mydict)
    for i,parm in enumerate(mydict):
        if i == n-1:
            output += f"{agg}(CASE WHEN {parm_label} = '{parm}' THEN {val} ELSE NULL END) AS {mydict[parm]}"
        else:
            output += f"{agg}(CASE WHEN {parm_label} = '{parm}' THEN {val} ELSE NULL END) AS {mydict[parm]},\n"
    parameters = "', '".join(mydict.keys())
    parameters = "'" + parameters + "'"
    return output, parameters

# Parameter Management

In [14]:
data = pd.read_csv("/Users/acyang/Documents/PythonProjects/5nm/Inline_Functional_Parms.csv")

In [15]:
data

,parm,short_name,key,category
0,SAILALL_H4_0p65Vto1p05V_PerfectYield,SailAll_Vnom,L1,Sail
1,SAILALL_H4_0p50Vto1p05V_PerfectYield,SailAll_Vmin,L1,Sail
2,SAIL1_H4_0p65Vto1p05V_PerfectYield,Sail1_Vnom,L2,Sail
3,SAIL1_H4_0p50Vto1p05V_PerfectYield,Sail1_Vmin,L2,Sail
4,SAIL2_H4_0p65Vto1p05V_PerfectYield,Sail2_Vnom,L2,Sail
...,...,...,...,...
89,PELE2_H4_Continuity_PerfectYield,Pele2_Continuity,L3,Other
90,PELE1_H4_ScanTest_Yield,Pele1_ScanTest,L3,Other
91,PELE2_H4_ScanTest_Yield,Pele2_ScanTest,L3,Other
92,PELE1_H4_MetalShorts_PerfectYield,Pele1_MetalShorts,L1,Other


In [16]:
schema = 'acayng'
new_qdat_table = 'inline_func_parms'

data.to_qdat_table(schema, new_qdat_table, if_exists='replace')

2023-11-09 16:12:05,410 INFO     ibmdata   : retrieved (1, 85) (rows, cols) of data in 0:00:00.353486 from CAYCE
2023-11-09 16:12:06,259 INFO     ibmdata   : retrieved (1, 85) (rows, cols) of data in 0:00:00.295774 from CAYCE
2023-11-09 16:12:07,120 INFO     ibmdata   : retrieved (1, 85) (rows, cols) of data in 0:00:00.301145 from CAYCE
2023-11-09 16:12:07,720 INFO     ibmdata   : retrieved (4, 58) (rows, cols) of data in 0:00:00.319876 from CAYCE
2023-11-09 16:12:08,660 INFO     ibmdata   : retrieved (1, 85) (rows, cols) of data in 0:00:00.295394 from CAYCE
2023-11-09 16:12:09,044 INFO     ibmdata   : successfully removed table acayng.inline_func_parms
2023-11-09 16:12:09,047 INFO     ibmdata   : dropped old table 'acayng.inline_func_parms'
2023-11-09 16:12:10,020 INFO     ibmdata   : retrieved (1, 85) (rows, cols) of data in 0:00:00.296103 from CAYCE
2023-11-09 16:12:10,589 INFO     ibmdata   : retrieved (4, 58) (rows, cols) of data in 0:00:00.294390 from CAYCE
2023-11-09 16:12:12,70

In [17]:
ibmdata.qdat.set_permissions(schema, new_qdat_table, '300mm_systems', read=True, write=True)
ibmdata.qdat.set_permissions(schema, new_qdat_table, '300mm_sysprc', read=True, write=True)

2023-11-09 16:12:15,766 INFO     ibmdata   : retrieved (1, 85) (rows, cols) of data in 0:00:00.302101 from CAYCE
2023-11-09 16:12:18,524 INFO     ibmdata   : retrieved (1, 85) (rows, cols) of data in 0:00:00.307284 from CAYCE


In [9]:
parm_df = ibmdata.qdat.query(f'select * from {schema}.{new_qdat_table}')

2023-11-07 19:54:23,709 INFO     ibmdata   : retrieved (94, 4) (rows, cols) of data in 0:00:00.185592 from CAYCE


In [10]:
L1_parms = parm_df[parm_df['key'] == 'L1'][['parm','short_name']].set_index('parm').T.to_dict('records')[0]

In [11]:
L1_parms

{'SAILALL_H4_0p65Vto1p05V_PerfectYield': 'SailAll_Vnom',
 'SAILALL_H4_0p50Vto1p05V_PerfectYield': 'SailAll_Vmin',
 'PELEALL_H4_K1KWMMarch90p75Vto1p05V_PerfectYield': 'PeleAll_1K_Vnom',
 'PELEALL_H4_K1KWMMarch90p655Vto1p05V_PerfectYield': 'PeleAll_1K_Vmin',
 'PELEALL_H4_M032March90p75Vto1p05V_PerfectYield': 'PeleAll_M032_Vnom',
 'PELEALL_H4_M032March90p655Vto1p05V_PerfectYield': 'PeleAll_M032_Vmin',
 'PELEALL_H4_M026VWBMarch90p75Vto1p05V_PerfectYield': 'PeleAll_M026VWB_Vnom',
 'PELEALL_H4_M026VWBMarch90p655Vto1p05V_PerfectYield': 'PeleAll_M026VWB_Vmin',
 'PELEALL_H4_M026NBEMarch90p75Vto1p05V_PerfectYield': 'PeleAll_M026NBE_Vnom',
 'PELEALL_H4_M026NBEMarch90p655Vto1p05V_PerfectYield': 'PeleAll_M026NBE_Vmin',
 'PELEALL_H4_M026VF6March90p75Vto1p05V_PerfectYield': 'PeleAll_M026VF6_Vnom',
 'PELEALL_H4_M026VF6March90p655Vto1p05V_PerfectYield': 'PeleAll_M026VF6_Vmin',
 'PELE1_H4_MetalShorts_PerfectYield': 'Pele1_MetalShorts',
 'PELE2_H4_MetalShorts_PerfectYield': 'Pele2_MetalShorts'}

In [5]:
def get_parm_df():
    schema = 'acayng'
    new_qdat_table = 'inline_func_parms'
    return ibmdata.qdat.query(f'select * from {schema}.{new_qdat_table}')

# SQL QUERIES

In [6]:
def wafer_agg_query(family_code, start, end, aggs, param_lst):
    if family_code == 'all':
        family_code = ALL_FAMILY_CODES
    else:
        family_code = f"'{family_code}'"
    return f"""
    SELECT 
        LEFT(wafer_id, 5) AS lot_id_base, lot_Id, wafer_Id, family_Code, 
        MIN(last_TestTimeStamp) AS earliest_test_date,
        {aggs}
    FROM 
        DMIW.ptilewaferfact ptwf
        INNER JOIN DMIW_SYSTEMS.testparm tp ON ptwf.testparmkey = tp.testparmkey
        INNER JOIN DMIW_SYSTEMS.testedwafer tw ON ptwf.testedwaferkey = tw.testedwaferkey
    WHERE DATE(last_testtimestamp) BETWEEN DATE('{start}') and DATE('{end}')
        AND parm_Label in ({param_lst})
        AND Tech_id = '5HPP'
        AND Level = 'H4'
        AND family_code IN ({family_code})
        AND weighted_mean IS NOT NULL
        AND abs(weighted_mean) < 1e25
    GROUP BY 
        lot_id, LEFT(wafer_id, 5), wafer_Id, family_Code
    ORDER BY
        lot_Id, wafer_Id
"""

In [7]:
def get_wafer_aggs(family_code, start, end, keys = 'all', category = 'all'):
    parm_df = get_parm_df()
    filtered_parm_df = parm_df
    if category != 'all':
        filtered_parm_df = filtered_parm_df[filtered_parm_df['category'].isin(category)]
    if keys != 'all':
        filtered_parm_df = filtered_parm_df[filtered_parm_df['key'].isin(keys)]
    parms_to_pull = filtered_parm_df[['parm','short_name']].set_index('parm').T.to_dict('records')[0]
    aggs, param_lst = generate_SQL_GroupBy(parms_to_pull, val = 'yield')
    df = ibmdata.isdw.query(wafer_agg_query(family_code, start, end, aggs, param_lst))

    return df.sort_values("earliest_test_date", ascending=False)

In [8]:
get_wafer_aggs('all', DEFAULT_START_DATE, DEFAULT_END_DATE, keys = 'all', category = 'all')

2023-11-25 03:52:06,711 INFO     ibmdata   : retrieved (94, 4) (rows, cols) of data in 0:00:00.479511 from CAYCE
2023-11-25 03:52:16,397 INFO     ibmdata   : retrieved (108, 99) (rows, cols) of data in 0:00:03.424485 from ISDW


,lot_id_base,lot_id,wafer_id,family_code,earliest_test_date,sailall_vnom,sailall_vmin,sail1_vnom,sail1_vmin,sail2_vnom,...,pele2_m026vf6_0p655v,pele2_m026vf6_0p75v,pele2_m026vf6_0p875v,pele2_m026vf6_1p05v,pele1_continuity,pele2_continuity,pele1_scantest,pele2_scantest,pele1_metalshorts,pele2_metalshorts
97,FZ513,FZ513.1,FZ513-25,M1,2023-11-22 21:29:51,97.46,93.67,97.67,97.67,97.77,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,FZ513,FZ513.1,FZ513-24,M1,2023-11-22 21:22:06,98.73,94.93,98.83,96.51,100.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,FZ513,FZ513.1,FZ513-23,M1,2023-11-22 21:15:01,94.93,93.67,95.34,95.34,96.66,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
94,FZ513,FZ513.1,FZ513-22,M1,2023-11-22 21:07:55,94.93,92.40,98.83,97.67,94.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
93,FZ513,FZ513.1,FZ513-21,M1,2023-11-22 21:00:53,97.46,93.67,97.67,95.34,100.00,...,57.77,93.330,98.88,100.00,100.00,100.0,100.00,100.00,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,FZ4YS,FZ4YS.1,FZ4YS-02,M1,2023-11-05 14:06:35,98.73,94.93,98.83,95.34,98.88,...,66.66,90.000,96.66,100.00,100.00,100.0,100.00,100.00,100.0,100.0
3,FZ4YS,FZ4YS.1,FZ4YS-01,M1,2023-11-05 13:58:02,92.30,89.74,94.18,93.02,95.55,...,62.22,92.220,97.77,98.88,98.83,100.0,98.83,100.00,100.0,100.0
1,FZ4FA,FZ4FA.1,FZ4FA-07,M1,2023-10-26 10:42:00,NaN,NaN,NaN,NaN,NaN,...,18.88,65.550,75.55,75.55,100.00,100.0,100.00,100.00,100.0,100.0
2,FZ4FA,FZ4FAB.1,FZ4FA-07,M1,2023-09-25 16:10:42,98.73,91.13,100.00,96.51,93.33,...,16.85,65.725,75.28,76.40,100.00,100.0,100.00,99.44,100.0,100.0


In [9]:
def get_lot_aggs(family_code, start, end, keys='all', category='all'):
    temp_agg_df = get_wafer_aggs(family_code, start, end, keys, category)
    temp_agg_df["earliest_test_date"] = temp_agg_df["earliest_test_date"].fillna(datetime.date(9999, 12, 30))
    
    groupby_ignore = ["lot_id_base", "lot_id", "family_code", "wafer_id"]
    groupby_dict = {c: 'mean' for c in temp_agg_df.columns if c not in groupby_ignore}
    groupby_dict.update({'earliest_test_date': 'min'})
    
    lot_df = (
        temp_agg_df.groupby(["lot_id_base", "lot_id", "family_code"])
        .agg(groupby_dict)
        .round(2)
        .sort_values("earliest_test_date", ascending=False)
        .reset_index()
    )
    lot_df["earliest_test_date"] = lot_df["earliest_test_date"].replace(datetime.date(9999, 12, 30), np.nan)
    
    return lot_df

In [12]:
data = get_lot_aggs('all', DEFAULT_START_DATE, DEFAULT_END_DATE, keys = 'all', category = 'all')


2023-11-25 03:53:51,918 INFO     ibmdata   : retrieved (94, 4) (rows, cols) of data in 0:00:00.473455 from CAYCE
2023-11-25 03:53:58,087 INFO     ibmdata   : retrieved (108, 99) (rows, cols) of data in 0:00:02.806553 from ISDW


In [13]:
data

,lot_id_base,lot_id,family_code,earliest_test_date,sailall_vnom,sailall_vmin,sail1_vnom,sail1_vmin,sail2_vnom,sail2_vmin,...,pele2_m026vf6_0p655v,pele2_m026vf6_0p75v,pele2_m026vf6_0p875v,pele2_m026vf6_1p05v,pele1_continuity,pele2_continuity,pele1_scantest,pele2_scantest,pele1_metalshorts,pele2_metalshorts
0,FZ513,FZ513.1,M1,2023-11-22 18:38:19,95.94,93.36,97.02,95.95,97.55,96.17,...,58.57,91.90,98.35,99.52,100.00,100.0,100.00,100.00,100.0,100.0
1,FZ519,FZ519.1,M1,2023-11-20 11:41:26,98.10,95.18,98.37,96.39,98.55,97.44,...,68.33,95.66,99.33,99.89,100.00,100.0,100.00,100.00,100.0,100.0
2,FZ50T,FZ50T.1,M1,2023-11-13 21:34:08,93.82,84.81,96.41,91.62,96.04,90.71,...,35.55,71.68,93.46,98.57,99.77,100.0,99.77,100.00,100.0,100.0
3,FZ4Z2,FZ4Z2.1,M1,2023-11-12 15:03:12,94.30,91.20,96.33,94.88,96.44,95.11,...,57.72,93.27,98.61,99.05,99.94,100.0,99.94,100.00,100.0,100.0
4,FZ4ZF,FZ4ZFA.1,M1,2023-11-11 14:29:51,93.92,81.77,97.55,90.11,95.66,89.33,...,47.66,83.33,98.22,99.78,100.00,100.0,100.00,100.00,100.0,100.0
5,FZ4ZF,FZ4ZF.1,M1,2023-11-10 23:43:40,94.17,77.72,97.90,88.60,95.55,86.22,...,45.55,81.99,98.44,99.55,100.00,100.0,100.00,100.00,100.0,100.0
6,FZ4YS,FZ4YS.1,M1,2023-11-05 13:58:02,96.57,91.25,97.90,94.76,97.44,95.55,...,56.33,87.44,96.22,99.44,99.65,100.0,99.65,100.00,100.0,100.0
7,FZ4FA,FZ4FAB.1,M1,2023-09-25 16:10:42,98.73,91.13,100.00,96.51,93.33,90.00,...,16.85,65.72,75.28,76.40,100.00,100.0,100.00,99.44,100.0,100.0
8,FZ4FA,FZ4FA.1,M1,2023-08-30 00:52:04,100.00,100.00,100.00,100.00,100.00,100.00,...,9.44,66.10,71.10,87.78,100.00,100.0,100.00,100.00,100.0,100.0


# Graphing

In [33]:
family_code = "all"
def plot_sail_graphs(n: int):
    w_df = get_wafer_aggs(family_code, DEFAULT_START_DATE, DEFAULT_END_DATE, keys = 'all', category = 'all')

    fig = go.Figure()

    g_title = "Sail Aggregate Yields"
    plot_params = (
        ("sailall_vmin", "Sail ALL 0.50 to 1.05", "#53cf4e", True),
        ("sailall_vnom", "Sail ALL 0.65 to 1.05", "#25572d", True),
        ("sail1_vmin", "Sail 1 0.50 to 1.05", "#b53024", "legendonly"),
        ("sail1_vnom", "Sail 1 0.65 to 1.05", "#e06234", "legendonly"),
        ("sail2_vmin", "Sail 2 0.50 to 1.05", "#242bbd", "legendonly"),
        ("sail2_vnom", "Sail 2 0.65 to 1.05", "#3667b5", "legendonly"),
    )
    parm_labels = [
            "lot_id_base",
            "earliest_test_date",
            "sailall_vmin",
            "sailall_vnom",
            "sail1_vmin",
            "sail1_vnom",
            "sail2_vmin",
            "sail2_vnom",
        ]
    lot_ids = w_df[parm_labels].dropna().sort_values("earliest_test_date", ascending=False).lot_id_base.unique()[0:n]
    w_df_temp = (
        w_df.loc[:, ~w_df.columns.duplicated()]
        .loc[w_df["lot_id_base"].isin(lot_ids)]
        .sort_values("earliest_test_date", ascending=True)
    )

    for param in plot_params:
        fig.add_trace(
            go.Box(
                y=w_df_temp[param[0]],
                x=w_df_temp.lot_id_base,
                name=param[1],
                marker=dict(color=param[2]),
                boxpoints="all",
                visible=param[3],
                text=w_df_temp.wafer_id,
                hovertemplate="<b>%{text}</b>" + "<br>%{y:.2f}%",
            )
        )

    fig.update_layout(
        title=g_title,
        yaxis_title="Yield (%)",
        boxmode="group",  # group together boxes of the different traces for each value of x
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )
    return fig


In [51]:
family_code = "all"
def plot_pele_graphs(selected_item: str, n: int):
    if not selected_item:
        raise PreventUpdate()

    if not n:
        raise PreventUpdate()
        
    w_df = get_wafer_aggs(family_code, DEFAULT_START_DATE, DEFAULT_END_DATE, keys = 'all', category = 'all')

    fig = go.Figure()

    if selected_item == "agg":
        g_title = "Pele Aggregate Yields"
        plot_params = (
            ("peleall_1k_vmin", "PeleALL 1K 0.655 to 1.05", "#53cf4e", True),
            ("peleall_1k_vnom", "PeleALL 1K 0.75 to 1.05", "#25572d", True),
            ("peleall_m032_vmin", "PeleAll M032 0.655 to 1.05", "#b53024", True),
            ("peleall_m032_vnom", "PeleAll M032 0.75 to 1.05", "#e06234", True),
            ("peleall_m026vwb_vmin", "PeleAll M026VWB 0.655 to 1.05", "#242bbd", True),
            ("peleall_m026vwb_vnom", "PeleAll M026VWB 0.75 to 1.05", "#3667b5", True),
            ("peleall_m026vf6_vmin", "PeleAll M026VF6 0.655 to 1.05", "#42c0d4", "legendonly"),
            ("peleall_m026vf6_vnom", "PeleAll M026VF6 0.75 to 1.05", "#298391", "legendonly"),
            ("peleall_m026nbe_vmin", "PeleAll M026NBE 0.655 to 1.05", "#a06ce6", "legendonly"),
            ("peleall_m026nbe_vnom", "PeleAll M026NBE 0.75 to 1.05", "#6232a1", "legendonly"),
        )

    elif selected_item == "base":
        g_title = "Pele Yields"
        plot_params = (
            ("pele1_1k_vmin", "Pele1 1K 0.655 to 1.05", "#53cf4e", "legendonly"),
            ("pele1_1k_vnom", "Pele1 1K 0.75 to 1.05", "#25572d", True),
            ("pele2_1k_vmin", "Pele2 1K 0.655 to 1.05", "#b53024", "legendonly"),
            ("pele2_1k_vnom", "Pele2 1K 0.75 to 1.05", "#e06234", True),
            ("pele1_m032_vmin", "Pele1 M032 0.655 to 1.05", "#242bbd", "legendonly"),
            ("pele1_m032_vnom", "Pele1 M032 0.75 to 1.05", "#3667b5", True),
            ("pele2_m032_vmin", "Pele2 M032 0.655 to 1.05", "#42c0d4", "legendonly"),
            ("pele2_m032_vnom", "Pele2 M032 0.75 to 1.05", "#298391", True),
            ("pele1_m026vwb_vmin", "Pele1 M026VWB 0.655 to 1.05", "#a06ce6", "legendonly"),
            ("pele1_m026vwb_vnom", "Pele1 M026VWB 0.75 to 1.05", "#6232a1", True),
            ("pele2_m026vwb_vmin", "Pele2 M026VWB 0.655 to 1.05", "#f5cf45", "legendonly"),
            ("pele2_m026vwb_vnom", "Pele2 M026VWB 0.75 to 1.05", "#e8a70e", True),
            ("pele1_m026vf6_vmin", "Pele1 M026VF6 0.655 to 1.05", "#f090ed", "legendonly"),
            ("pele1_m026vf6_vnom", "Pele1 M026VF6 0.75 to 1.05", "#d121cc", "legendonly"),
            ("pele2_m026vf6_vmin", "Pele2 M026VF6 0.655 to 1.05", "#eda17b", "legendonly"),
            ("pele2_m026vf6_vnom", "Pele2 M026VF6 0.75 to 1.05", "#ed621c", "legendonly"),
            ("pele1_m026nbe_vmin", "Pele1 M026NBE 0.655 to 1.05", "#bced1c", "legendonly"),
            ("pele1_m026nbe_vnom", "Pele1 M026NBE 0.75 to 1.05", "#86e31b", "legendonly"),
            ("pele2_m026nbe_vmin", "Pele2 M026NBE 0.655 to 1.05", "#e884b6", "legendonly"),
            ("pele2_m026nbe_vnom", "Pele2 M026NBE 0.75 to 1.05", "#e01078", "legendonly"),
        )
    parm_labels = [
            "lot_id_base",
            "earliest_test_date",
            "peleall_1k_vnom",
            "peleall_1k_vmin",
            "peleall_m032_vnom",
            "peleall_m032_vmin",
            "peleall_m026vwb_vnom",
            "peleall_m026vwb_vmin",
            "peleall_m026vf6_vnom",
            "peleall_m026vf6_vmin",
            "peleall_m026nbe_vnom",
            "peleall_m026nbe_vmin",
        ]
    lot_ids = w_df[parm_labels].dropna().sort_values("earliest_test_date", ascending=False).lot_id_base.unique()[0:n]
    w_df_temp = (
        w_df.loc[:, ~w_df.columns.duplicated()]
        .loc[w_df["lot_id_base"].isin(lot_ids)]
        .sort_values("earliest_test_date", ascending=True)
    )

    for param in plot_params:
        fig.add_trace(
            go.Box(
                y=w_df_temp[param[0]],
                x=w_df_temp.lot_id_base,
                name=param[1],
                marker=dict(color=param[2]),
                boxpoints="all",
                visible=param[3],
                text=w_df_temp.wafer_id,
                hovertemplate="<b>%{text}</b>" + "<br>%{y:.2f}%",
            )
        )

    fig.update_layout(
        title=g_title,
        yaxis_title="Yield (%)",
        boxmode="group",  # group together boxes of the different traces for each value of x
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    )
    return fig

In [34]:
plot_sail_graphs(3)

2023-11-17 17:03:46,116 INFO     ibmdata   : retrieved (94, 4) (rows, cols) of data in 0:00:00.187773 from CAYCE
2023-11-17 17:03:49,222 INFO     ibmdata   : retrieved (73, 99) (rows, cols) of data in 0:00:00.901039 from ISDW


In [52]:
plot_pele_graphs('base',5)

2023-11-17 17:39:38,852 INFO     ibmdata   : retrieved (94, 4) (rows, cols) of data in 0:00:00.187934 from CAYCE
2023-11-17 17:39:41,417 INFO     ibmdata   : retrieved (73, 99) (rows, cols) of data in 0:00:00.945735 from ISDW


# PlayGround

In [ ]:
# Pull Wafer Level Data
aggs = generate_SQL_GroupBy(params, val = 'yield')
def wafer_agg_query(family_code, start, end):
    if family_code == 'all':
        family_code = "'M1'"
    else:
        family_code = f"'{family_code}'"
    return f"""
    SELECT 
        lot_Id, LEFT(wafer_id, 5) AS lot_id_base, wafer_Id, family_Code, 
        MIN(last_TestTimeStamp) AS earliest_test_date,
        {aggs}
    FROM 
        DMIW.ptilewaferfact ptwf
        INNER JOIN DMIW_SYSTEMS.testparm tp ON ptwf.testparmkey = tp.testparmkey
        INNER JOIN DMIW_SYSTEMS.testedwafer tw ON ptwf.testedwaferkey = tw.testedwaferkey
    WHERE DATE(last_testtimestamp) BETWEEN DATE('{start}') and DATE('{end}')
        AND parm_Label in ({parameters})
        AND Tech_id = '5HPP'
        AND Level = 'H4'
        --AND family_code = {family_code}
        AND weighted_mean IS NOT NULL
        AND abs(weighted_mean) < 1e25
    GROUP BY 
        lot_id, LEFT(wafer_id, 5), wafer_Id, family_Code
    ORDER BY
        lot_Id, wafer_Id
"""

In [11]:
# Pull Chip Level Data
QUERY = f"""
SELECT
    lot_Id, LEFT(wafer_id, 5) AS lot_id_base, wafer_Id, normalized_TestX AS chipX, normalized_TestY AS chipY,
    family_Code, last_TestTimeStamp AS date, tw.calcdefs, parm_Label AS PARAMETER, parmValue, parmYield AS yield,
    radius_Center_5, quadrant
FROM
    DMIW.ChipParmFactR cpfr
    INNER JOIN DMIW_SYSPRC.TestParm tp ON cpfr.testparmkey = tp.testparmkey 
    INNER JOIN DMIW_SYSPRC.Geography g ON cpfr.geographykey = g.geographykey 
    INNER JOIN DMIW_SYSPRC.testedwafer tw ON cpfr.testedwaferkey = tw.testedwaferkey
WHERE ( Last_test_date >= (current date - 600 days))
    AND parm_Label in ({parameters})
    AND Tech_id = '5HPP'
    AND Level = 'H4'
    AND cpfr.parmValue IS NOT NULL
    AND abs(parmValue) < 1e25
ORDER BY
    lot_Id, wafer_Id, normalized_TestX, normalized_TestY, parm_Label, tw.calcdefs
"""

In [19]:
# Chip Level Data aggregated by wafer

aggs = generate_SQL_GroupBy(params)
def wafer_agg_query(family_code, start, end):
    if family_code == 'all':
        family_code = "'M1'"
    else:
        family_code = f"'{family_code}'"
    return f"""
    SELECT
        lot_Id, LEFT(wafer_id, 5) AS lot_id_base, wafer_Id, family_Code, 
        MIN(last_TestTimeStamp) AS earliest_test_date,
        {aggs}
    FROM
        DMIW.ChipParmFactR cpfr
        INNER JOIN DMIW_SYSPRC.TestParm tp ON cpfr.testparmkey = tp.testparmkey 
        INNER JOIN DMIW_SYSPRC.Geography g ON cpfr.geographykey = g.geographykey 
        INNER JOIN DMIW_SYSPRC.testedwafer tw ON cpfr.testedwaferkey = tw.testedwaferkey
    WHERE DATE(last_testtimestamp) BETWEEN DATE('{start}') and DATE('{end}')
        AND parm_Label in ({parameters})
        AND Tech_id = '5HPP'
        AND Level = 'H4'
        --AND family_code = {family_code}
        AND cpfr.parmValue IS NOT NULL
        AND abs(parmValue) < 1e25
    GROUP BY 
        lot_id, LEFT(wafer_id, 5), wafer_Id, family_Code
    ORDER BY
        lot_Id, wafer_Id
    """

In [20]:
df = ibmdata.isdw.query(QUERY)

2023-10-11 14:53:23,351 INFO     ibmdata   : retrieved (99735, 21) (rows, cols) of data in 0:00:47.273925 from ISDW


In [21]:
df

,lot_id,wafer_id,family_code,date,level,parameter,lsl,target,usl,n,...,stddev,min,ptile05,ptile10,ptile25,ptile50,ptile75,ptile90,ptile95,max
0,AZDPB.1,AZDPB-04,5A,2022-06-10 00:00:00,M1,PL032S_FET_PD_DIBL_M1,35.799999,NaN,42.500000,13,...,8.056886,37.174000,37.174000,37.228001,41.092999,44.945000,47.407001,59.527000,62.904999,62.904999
1,AZDPB.1,AZDPB-04,5A,2022-06-10 00:00:00,M1,PL032S_FET_PD_Iboff_M1,1.840000,NaN,10.510000,13,...,1.713160,-0.260000,-0.260000,2.625000,2.960000,3.430000,4.905000,5.770000,6.453000,6.453000
2,AZDPB.1,AZDPB-04,5A,2022-06-10 00:00:00,M1,PL032S_FET_PD_Idoff_M1,9.390000,NaN,764.799988,13,...,217.432434,27.927999,27.927999,56.105000,76.178001,94.665001,142.707993,432.018005,822.534973,822.534973
3,AZDPB.1,AZDPB-04,5A,2022-06-10 00:00:00,M1,PL032S_FET_PD_Idsat_M1,76.989998,NaN,123.000000,13,...,5.064284,76.190002,76.190002,82.209999,82.860001,86.720001,88.940002,92.570000,96.370003,96.370003
4,AZDPB.1,AZDPB-04,5A,2022-06-10 00:00:00,M1,PL032S_FET_PD_Iginv_M1,0.010000,NaN,0.050000,13,...,4.565075,21.305000,21.305000,23.768000,26.358000,29.100000,30.243000,33.419998,39.373001,39.373001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99730,FZ4YS.1,FZ4YS-10,M1,2023-10-11 09:26:47,M1,S026_D9_SP_FET_PL032S_M1_4b_ult_EDGE_corner_PU...,NaN,NaN,NaN,13,...,0.000010,0.000063,0.000063,0.000064,0.000075,0.000083,0.000086,0.000091,0.000092,0.000092
99731,FZ4YS.1,FZ4YS-10,M1,2023-10-11 09:26:47,M1,S026_D9_SP_FET_PL032S_M1_4b_ult_EDGE_corner_PU...,NaN,NaN,NaN,13,...,0.024849,-0.379428,-0.379428,-0.377880,-0.364498,-0.346008,-0.340302,-0.316162,-0.290038,-0.290038
99732,FZ4YS.1,FZ4YS-10,M1,2023-10-11 09:26:47,M1,S026_D9_SP_FET_PL032S_M1_4b_ult_EDGE_corner_PU...,NaN,NaN,NaN,13,...,0.020107,-0.459234,-0.459234,-0.450996,-0.433684,-0.422734,-0.409316,-0.398450,-0.387640,-0.387640
99733,FZ4YS.1,FZ4YS-10,M1,2023-10-11 09:26:47,M1,S026_D9_SP_FET_PL032S_M1_4b_ult_EDGE_corner_PU...,NaN,NaN,NaN,13,...,0.026951,-0.332146,-0.332146,-0.331390,-0.311664,-0.303258,-0.296718,-0.275394,-0.227752,-0.227752


In [14]:
df1 = df[df['lot_id_base'] == 'AZDRB']

In [20]:
groups = {'g1':['01','02'], 'g2':['11','13'], 'g3':['21','22'], 'g4':['05','10','14','16'], 'g5':['06'], 'g6':['17','19'], 'g7':['18','23']}

In [48]:
def labeling(x):
    wid = x[-2:]
    if wid in groups['g1']:
        return 'POR'
    elif wid in groups['g2']:
        return 'POR'
    elif wid in groups['g3']:
        return 'POR'
    elif wid in groups['g4']:
        return 'POR'
    elif wid in groups['g5']:
        return 'POR'
    elif wid in groups['g6']:
        return 'MOLopt2'
    elif wid in groups['g7']:
        return 'MOLopt2'
    else:
        return 'other'

In [49]:
df1['MOLANN'] = df1['wafer_id'].map(lambda x: labeling(x))

/var/folders/tt/zcxx79vj73546c33_7xnq7yw0000gn/T/ipykernel_22972/409344166.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['MOLANN'] = df1['wafer_id'].map(lambda x: labeling(x))


In [51]:
df1.to_csv('~/Downloads/asdfas.csv')

In [7]:
# Chip Level Data aggregated by Zone and Qudrant (Drilldown)
QUERY = f"""
WITH cte AS(
    SELECT
        lot_Id, LEFT(wafer_id, 5) AS lot_id_base, wafer_Id, family_Code, radius_Center_5 AS ER_Zone,
        MIN(last_TestTimeStamp) AS earliest_test_date,
        {aggs}
    FROM
        DMIW.ChipParmFactR cpfr
        INNER JOIN DMIW_SYSPRC.TestParm tp ON cpfr.testparmkey = tp.testparmkey 
        INNER JOIN DMIW_SYSPRC.Geography g ON cpfr.geographykey = g.geographykey 
        INNER JOIN DMIW_SYSPRC.testedwafer tw ON cpfr.testedwaferkey = tw.testedwaferkey
    WHERE ( Last_test_date >= (current date - 300 days))
        AND parm_Label in ({parameters})
        AND Tech_id = '5HPP'
        AND Level = 'H4'
        AND cpfr.parmValue IS NOT NULL
        AND abs(parmValue) < 1e25
        AND wafer_id IN ('AZDU1-01')
    GROUP BY lot_id, LEFT(wafer_id, 5), wafer_Id, family_Code, radius_Center_5
    ORDER BY
        lot_Id, wafer_Id
)
SELECT *
FROM cte
"""

In [45]:
df = ibmdata.isdw.query(QUERY)

2023-07-28 15:26:34,129 INFO     ibmdata   : retrieved (5, 44) (rows, cols) of data in 0:00:00.740153 from ISDW


In [46]:
df

,lot_id,lot_id_base,wafer_id,family_code,er_zone,earliest_test_date,pele1_m026vwb_0p655v,pele1_m026vwb_0p75v,pele1_m026vwb_1p05v,pele2_m026vwb_0p655v,...,pele2_m032_1p05v,sail1_vmin,sail1_vnom,sail1_skiptest_0p65v,sail1_skiptest_0p50v,sail1_skiptest_0p75v,sail1_skiptest_1p05v,sail1_continuity,sailall_vmin,sailall_vnom
0,AZDU1D.1,AZDU1,AZDU1-01,5A,A,2023-02-19 20:28:31,100.0,100.0,100.000000,100.0,...,100.000000,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.000000,100.0
1,AZDU1D.1,AZDU1,AZDU1-01,5A,B,2023-02-19 20:28:31,100.0,100.0,100.000000,100.0,...,100.000000,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.000000,100.0
2,AZDU1D.1,AZDU1,AZDU1-01,5A,C,2023-02-19 20:28:31,100.0,100.0,100.000000,100.0,...,100.000000,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.000000,100.0
3,AZDU1D.1,AZDU1,AZDU1-01,5A,D,2023-02-19 20:28:31,100.0,100.0,96.153846,100.0,...,100.000000,100.0,100.0,100.0,100.0,100.0,100.0,100.0,100.000000,100.0
4,AZDU1D.1,AZDU1,AZDU1-01,5A,E,2023-02-19 20:28:31,96.0,96.0,92.000000,100.0,...,96.296296,100.0,100.0,100.0,100.0,100.0,100.0,100.0,95.454545,100.0


In [47]:
mydict = {
'A':'A',
'B':'B',
'C':'C',
'D':'D',
'E':'E'}
aggs = generate_SQL_GroupBy(mydict, parm_label = 'radius_Center_5', val = 'parmYield', agg = 'AVG')

In [48]:
aggs

"AVG(CASE WHEN radius_Center_5 = 'A' THEN parmYield ELSE NULL END) AS A,\nAVG(CASE WHEN radius_Center_5 = 'B' THEN parmYield ELSE NULL END) AS B,\nAVG(CASE WHEN radius_Center_5 = 'C' THEN parmYield ELSE NULL END) AS C,\nAVG(CASE WHEN radius_Center_5 = 'D' THEN parmYield ELSE NULL END) AS D,\nAVG(CASE WHEN radius_Center_5 = 'E' THEN parmYield ELSE NULL END) AS E"

In [49]:
QUERY = f"""
WITH cte AS(
    SELECT
        lot_Id, LEFT(wafer_id, 5) AS lot_id_base, wafer_Id, family_Code,
        MIN(last_TestTimeStamp) AS earliest_test_date, parm_label,
        {aggs}
    FROM
        DMIW.ChipParmFactR cpfr
        INNER JOIN DMIW_SYSPRC.TestParm tp ON cpfr.testparmkey = tp.testparmkey 
        INNER JOIN DMIW_SYSPRC.Geography g ON cpfr.geographykey = g.geographykey 
        INNER JOIN DMIW_SYSPRC.testedwafer tw ON cpfr.testedwaferkey = tw.testedwaferkey
    WHERE ( Last_test_date >= (current date - 300 days))
        AND parm_Label in ({parameters})
        AND Tech_id = '5HPP'
        AND Level = 'H4'
        AND cpfr.parmValue IS NOT NULL
        AND abs(parmValue) < 1e25
        AND wafer_id IN ('AZDU1-01')
    GROUP BY lot_id, LEFT(wafer_id, 5), wafer_Id, family_Code, parm_label
    ORDER BY
        lot_Id, wafer_Id
)
SELECT *
FROM cte
"""

In [18]:
df = ibmdata.isdw.query(QUERY)

2023-10-11 14:52:34,571 INFO     ibmdata   : retrieved (99735, 21) (rows, cols) of data in 0:00:47.852796 from ISDW


In [51]:
df

,lot_id,lot_id_base,wafer_id,family_code,earliest_test_date,parm_label,a,b,c,d,e
0,AZDU1D.1,AZDU1,AZDU1-01,5A,2023-02-20 01:32:06,Pele1~H4~K1KWMMarch90p60,0.000000,0.0,33.333333,80.769231,92.000000
1,AZDU1D.1,AZDU1,AZDU1-01,5A,2023-02-20 01:32:06,Pele1~H4~K1KWMMarch90p655,100.000000,50.0,86.666667,100.000000,96.000000
2,AZDU1D.1,AZDU1,AZDU1-01,5A,2023-02-20 01:32:06,Pele1~H4~K1KWMMarch90p75,100.000000,100.0,100.000000,100.000000,100.000000
3,AZDU1D.1,AZDU1,AZDU1-01,5A,2023-02-20 01:32:06,Pele1~H4~K1KWMMarch91p05,100.000000,100.0,100.000000,100.000000,100.000000
4,AZDU1D.1,AZDU1,AZDU1-01,5A,2023-02-20 01:32:06,Pele1~H4~M026March90p85,0.000000,0.0,0.000000,0.000000,0.000000
5,AZDU1D.1,AZDU1,AZDU1-01,5A,2023-02-20 01:32:06,Pele1~H4~M026March90p90,0.000000,0.0,0.000000,0.000000,0.000000
6,AZDU1D.1,AZDU1,AZDU1-01,5A,2023-02-20 01:32:06,Pele1~H4~M026March91p05,0.000000,0.0,0.000000,15.384615,68.000000
7,AZDU1D.1,AZDU1,AZDU1-01,5A,2023-02-20 01:32:06,Pele1~H4~M026VWBMarch90p655,100.000000,100.0,100.000000,100.000000,96.000000
8,AZDU1D.1,AZDU1,AZDU1-01,5A,2023-02-20 01:32:06,Pele1~H4~M026VWBMarch90p75,100.000000,100.0,100.000000,100.000000,96.000000
9,AZDU1D.1,AZDU1,AZDU1-01,5A,2023-02-20 01:32:06,Pele1~H4~M026VWBMarch91p05,100.000000,100.0,100.000000,96.153846,92.000000


# Correlation Finder

In [9]:
DAYSBACK = 600
SEARCH = 'PL032'
#SEARCH = 'M032'
QUERY = f"""
SELECT 
    lot_id, wafer_id, family_code, date, level, parm_label as parameter,
    weighted_mean as mean
FROM
    DMIW.ptilewaferfact ptwf,
    DMIW_SYSPRC.testparm tp,
    (SELECT 
        testedWaferKey, lot_Id, last_fab_lot_id, wafer_Id, 
        family_Code, cur_Lot_Grade as lot_Grade, 
        last_TestTimeStamp as date, level
        FROM 
          DMIW_SYSPRC.testedwafer tw
        WHERE 
            ( Last_test_date >= (current date - {DAYSBACK} days) AND Tech_id = '5HPP' AND Calcdefs NOT IN ('PLE11', 'PLE21'))
    ) tw
WHERE 
    tw.testedwaferkey = ptwf.testedwaferkey 
    AND tp.testparmkey = ptwf.testparmkey
    AND weighted_mean IS NOT NULL 
    AND abs(weighted_mean) < 1e25
    AND ( ucase(tp.parm_Label) LIKE '%{SEARCH}%' )  
ORDER BY 
    lot_Id, wafer_Id, parm_Label, level
"""
a = ibmdata.isdw.query(QUERY)

2023-11-06 09:51:48,549 INFO     ibmdata   : retrieved (129569, 7) (rows, cols) of data in 0:00:36.220997 from ISDW


In [11]:
a.columns

Index(['lot_id', 'wafer_id', 'family_code', 'date', 'level', 'parameter',
       'mean'],
      dtype='object')

In [56]:
a

,lot_id,wafer_id,family_code,date,level,parameter,mean
0,AZDPB.1,AZDPB-04,5A,2022-06-08,M1,FC02_SRSh_D01_M032_WL2BL~VBD_M1,-9.800000e+00
1,AZDPB.1,AZDPB-04,5A,2022-06-08,M1,FC02_SRSh_D02_M032_WL2BL_STI~VBD_M1,-9.800000e+00
2,AZDPB.1,AZDPB-04,5A,2022-06-08,M1,FC03_SRSh_D01_M032_WL2ND_Vbd~VBD_M1,-7.692308e-01
3,AZDPB.1,AZDPB-04,5A,2022-06-08,M1,FC03_SRSh_D02_M032_GND2ND_Vbd~VBD_M1,-4.230769e+00
4,AZDPB.1,AZDPB-04,5A,2022-06-10,M1,M032_40FIN_A_PD_Idsat_M1,2.773077e+02
...,...,...,...,...,...,...,...
123850,FZ4YS.1,FZ4YS-10,M1,2023-10-15,M1,S025_D5_SP_LKG_M032_M2_w_strap_96Kb_lat~140Ipw_M1,-1.911100e-08
123851,FZ4YS.1,FZ4YS-10,M1,2023-10-15,M1,S025_D5_SP_LKG_M032_M2_w_strap_96Kb_lat~140Isb...,1.915461e-08
123852,FZ4YS.1,FZ4YS-10,M1,2023-10-15,M1,S025_D5_SP_LKG_M032_M2_w_strap_96Kb_lat~140Isb...,-1.911093e-08
123853,FZ4YS.1,FZ4YS-10,M1,2023-10-15,M1,S025_D5_SP_LKG_M032_M2_w_strap_96Kb_lat~140Iss_M1,6.846154e-14


In [32]:
a[a.duplicated()]

,lot_id,wafer_id,family_code,date,level,parameter,mean


In [10]:
b = a.drop_duplicates().pivot(index=['lot_id','wafer_id','family_code','level','date'], columns=['parameter'],values=['mean'])

In [11]:
b = b.droplevel(0,axis=1).reset_index()

In [35]:
b

parameter,lot_id,wafer_id,family_code,level,date,P032_PD_VTU_GRAND_LT,P032_PD_VtMM_LT,P032_PD_VtMM_M5,P032_PD_Vtsat_Delta_LT,P032_PG_VTU_GRAND_LT,...,P032_Vtmm_PU_R_8_Idsat_LT,P032_Vtmm_PU_R_8_Vtlin_LT,P032_Vtmm_PU_R_8_Vtsat_LT,P032_Vtmm_PU_R_9_Idlin_LT,P032_Vtmm_PU_R_9_Idsat_LT,P032_Vtmm_PU_R_9_Vtlin_LT,P032_Vtmm_PU_R_9_Vtsat_LT,P032_Vtmm_PU_Vtsat_LT,P032_Vtmm_PU_Vtsat_L_LT,P032_Vtmm_PU_Vtsat_R_LT
0,AZDSW.1,AZDSW-06,5A,LT,2022-09-22,78.400002,29.900000,NaN,22.496536,73.900002,...,NaN,NaN,396.210663,NaN,NaN,NaN,394.727478,392.181061,403.748627,400.885773
1,AZDSW.1,AZDSW-07,5A,LT,2022-09-22,73.199997,29.200001,NaN,24.154289,71.199997,...,NaN,NaN,396.289032,NaN,NaN,NaN,395.621552,396.303833,394.292938,393.474670
2,AZDSW.1,AZDSW-10,5A,LT,2022-09-22,73.099998,28.799999,NaN,24.548111,70.599998,...,NaN,NaN,388.006805,NaN,NaN,NaN,395.795685,390.879517,401.712280,388.955231
3,AZFNS.1,AZFNS-01,5A,M5,2023-02-01,NaN,NaN,30.799999,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AZFNS.1,AZFNS-03,5A,M5,2023-02-01,NaN,NaN,38.099998,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,AZFNS.1,AZFNS-09,5A,M5,2023-02-01,NaN,NaN,33.299999,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,AZFNS.1,AZFNS-17,5A,M5,2023-02-01,NaN,NaN,28.600000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,AZFNS.1,AZFNS-18,5A,M5,2023-02-01,NaN,NaN,33.700001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,AZFNS.1,AZFNS-23,5A,M5,2023-02-01,NaN,NaN,29.600000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AZFW5.1,AZFW5-16,5A,LT,2023-03-21,NaN,NaN,NaN,NaN,NaN,...,27.762928,458.877594,417.145660,5.985572,27.497942,464.511292,423.192352,NaN,NaN,NaN


In [12]:
b = b.groupby(['lot_id','wafer_id','family_code','level']).mean(numeric_only=True).reset_index()

In [79]:
b.to_csv("~/Downloads/sfdaf.csv")

In [13]:
parms = list(b.columns[4:])

In [48]:
df = get_wafer_aggs('all', DEFAULT_START_DATE, DEFAULT_END_DATE, ['L1'])
df['1k_Vmin_delta'] = df['peleall_1k_vnom'] - df['peleall_1k_vmin']
df['m032_Vmin_delta'] = df['peleall_m032_vnom'] - df['peleall_m032_vmin']

2023-11-06 10:20:24,160 INFO     ibmdata   : retrieved (94, 4) (rows, cols) of data in 0:00:00.187670 from CAYCE
2023-11-06 10:20:25,945 INFO     ibmdata   : retrieved (13, 19) (rows, cols) of data in 0:00:00.277224 from ISDW


In [49]:
final = df[['wafer_id','1k_Vmin_delta', 'm032_Vmin_delta']].merge(b, how='inner', on='wafer_id')

In [67]:
final[['wafer_id','1k_Vmin_delta', 'm032_Vmin_delta','PL032S_Gamma_M5']]

,wafer_id,1k_Vmin_delta,m032_Vmin_delta,PL032S_Gamma_M5
0,FZ4YS-10,0.00,0.00,NaN
1,FZ4YS-10,0.00,0.00,NaN
2,FZ4YS-01,2.56,5.13,NaN
3,FZ4YS-01,2.56,5.13,NaN
4,FZ4FA-01,NaN,NaN,NaN
5,FZ4FA-01,NaN,NaN,NaN


In [51]:
from scipy.stats import spearmanr
import numpy as np
import plotly.express as px

In [55]:
output = []
y_var = '1k_Vmin_delta'
#y_var = 'm032_Vmin_delta'
for p in parms:
    data = final[[y_var,p]]
    data = data.dropna()
    if not data.empty and data.shape[0] > 10:
        corr, _ = spearmanr(data[p],data[y_var])
        output.append((p,corr))

In [56]:
results = sorted(output, key= lambda x: np.abs(x[1]), reverse = True)

In [57]:
results

[]

In [50]:
from pptx import Presentation
from pptx.util import Inches
from IPython.display import Image

In [68]:
prs = Presentation()
for r in results[:100]:
    fig = px.scatter(final, x=r[0], y=y_var, color = 'lot_id')
    fig.write_image("/Users/acyang//Downloads/fig.png")
    slide = prs.slides.add_slide(prs.slide_layouts[6])
    slide.shapes.add_picture("/Users/acyang//Downloads/fig.png",Inches(0), Inches(0))
prs.save('/Users/acyang/Downloads/test.pptx')

/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_comm

/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_comm

/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_comm

/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_comm

In [59]:
fig = px.scatter(final, x='PL032S_LVTP_ISB_M1', y='1k_Vmin_delta', color = 'lot_id')
fig.show()

/usr/local/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1641: DeprecationWarning:

np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)

